# CSE258 HW1

In [274]:
import numpy as np
import urllib
import scipy.optimize
import random
import csv
from sklearn.metrics import mean_squared_error
from sklearn import svm
from math import exp
from math import log

## Regression Q1

In [63]:
def parseData(fname):
    for l in urllib.urlopen(fname):
        yield eval(l)

print "Reading data..."
data = list(parseData("http://jmcauley.ucsd.edu/cse255/data/beer/beer_50000.json"))
print "done"

Reading data...
done


In [275]:
x1 = [[1,i['review/timeStruct']['year']] for i in data]

y = [i['review/overall'] for i in data]

#### We first train a predictor with linear regression as below:

#### $'review/overall' = \theta_0 + \theta_1 * 'year'$

#### The fitted values are $\theta_0 = -3.917e+01, \theta_1 = 2.1438e-02$

In [276]:
result1 = np.linalg.lstsq(x1,y)
print result1[0]
MSE1 = result1[1]/50000
print MSE1

[ -3.91707489e+01   2.14379786e-02]
[ 0.49004382]


## Regression Q2
#### Then we use a second-order polynomial predictor instead of linear one:
$'review/overall' = \theta_0 + \theta_1 * 'year' + \theta_2 * 'year'$
#### The MSE in Q1 is 0.49004382. While here we have MSE as 0.49003734, which is only little better. Actually, 'review/overall' can depends little on one-dimension feature, 'year', since we see that the beers in the same 'year' may varies a lot in 'review/overall'. So our improvement is little.

In [283]:
x2= []
for feat in x1:
    newFeat = feat[:]
    newFeat.append(feat[1]**2)
    x2.append(newFeat)

In [285]:
result2 = np.linalg.lstsq(x2,y)
print result2[0]
MSE2 = result2[1]/50000
print MSE2

[ -2.32112075e+02   2.13653648e-01  -4.78731119e-05]
[ 0.49004374]


## Regression Q3

In [339]:
with open('winequality-white.csv', 'rb') as f:
    reader = csv.reader(f, delimiter=';')
    wine = []
    for row in reader:
        wine.append(row)
for i in range(1,len(wine)):
    for j in range(0,len(wine[i])):
        wine[i][j] = float(wine[i][j])
print wine[0]
wine = wine[1:]
train = np.array(wine[0:len(wine)/2])
test = np.array(wine[len(wine)/2:len(wine)])

['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol', 'quality']


#### The fitted coefficients $\theta = [\theta_0, \theta_1,...]$ & train data MSE (0.6023) is calculated as below:

In [340]:
train_x = np.concatenate((np.ones((len(train),1)), train[:,0:-1]), axis=1)
train_y = train[:,-1]

In [341]:
result3 = np.linalg.lstsq(train_x,train_y)
MSE3 = result3[1]/len(train)
print result3[0]
print MSE3

[  2.56420279e+02   1.35421303e-01  -1.72994866e+00   1.02651152e-01
   1.09038568e-01  -2.76775146e-01   6.34332169e-03   3.85023977e-05
  -2.58652809e+02   1.19540566e+00   8.33006285e-01   9.79304353e-02]
[ 0.6023075]


#### The MSE on the test data is shown below:

In [342]:
test_x = np.concatenate((np.ones((len(test),1)), test[:,0:-1]), axis=1)
test_y = test[:,-1]

In [343]:
mean_squared_error(np.dot(test_x, result3[0]), test_y)

0.56245713031281874

## Regression Q4
#### The MSEs of all 11 ablation experiments are calculated below:

In [344]:
MSE_abl = []
for i in range(1,12):
    # get ablation feature data
    train_abl = np.delete(train_x, i, axis=1)
    test_abl = np.delete(test_x, i, axis=1)
    # training
    fit = np.linalg.lstsq(train_abl,train_y)
    # mse
    MSE_abl.append(mean_squared_error(np.dot(test_abl, fit[0]), test_y))
print MSE_abl

[0.55911341437613271, 0.59638485016151266, 0.56222170281155925, 0.55362506396744637, 0.56262926648130007, 0.55614081792995385, 0.56242900546920438, 0.54472655346615018, 0.5595666263819955, 0.55734634998788479, 0.57321474355822255]


#### Based on the test MSEs, we see 'density' provides the least additional information while 'volatile acidity' provides the most.

### Classification Q5

#### Under C=0.8, I have accuracy on train and test data as 89.91% and 69.86%.

In [345]:
train_lab = map(lambda x : 1 if x>5 else 0, train_y)
test_lab = map(lambda x : 1 if x>5 else 0, test_y)
clf = svm.SVC(C=0.8)
clf.fit(train_x, train_lab)

# prediction with classifier
train_pred = clf.predict(train_x)
test_pred = clf.predict(test_x)

In [346]:
train_pair = np.vstack((np.array(train_lab), train_pred))
test_pair = np.vstack((np.array(test_lab), test_pred))
train_correct = filter(lambda x : x[0] == x[1], train_pair.T)
test_correct = filter(lambda x : x[0] == x[1], test_pair.T)

In [347]:
train_accuracy = len(train_correct) * 1. /len(train_x)
test_accuracy = len(test_correct) * 1. /len(test_x)
print train_accuracy, test_accuracy

0.899142507146 0.698652511229


### Classification Q6

#### The log-likelihood after convergence is -1383.18, and the accuract of the resulting model is 76.68%

In [348]:
def inner(x,y):
    return sum([x[i]*y[i] for i in range(len(x))])

def sigmoid(x):
    return 1.0 / (1 + exp(-x))

In [349]:
# NEGATIVE Log-likelihood
def f(theta, X, y, lam):
    loglikelihood = 0
    for i in range(len(X)):
        logit = inner(X[i], theta)
        loglikelihood -= log(1 + exp(-logit))
        if not y[i]:
            loglikelihood -= logit
    for k in range(len(theta)):
        loglikelihood -= lam * theta[k]*theta[k]
    print "ll =", loglikelihood
    return -loglikelihood

In [360]:
# NEGATIVE Derivative of log-likelihood
def fprime(theta, X, y, lam):
    dl = [0.0]*len(theta)
    for i in range(len(X)):
        xi = sigmoid(inner(X[i], theta))
    # Fill in code for the derivative
        for j in range(len(dl)):
            dl[j] += X[i][j] * (1.0 - xi)
            if not y[i]:
                dl[j] -= X[i][j]
    dl -= lam * 2.0 * theta
    # Negate the return value since we're doing gradient *ascent*
    return numpy.array([-x for x in dl])

In [361]:
# If we wanted to split with a validation set:
#X_valid = X[len(X)/2:3*len(X)/4]
#X_test = X[3*len(X)/4:]

# Use a library function to run gradient descent (or you can implement yourself!)
theta,l,info = scipy.optimize.fmin_l_bfgs_b(f, [0]*len(train_x[0]), fprime, args = (train_x, train_lab, 1.0))
print "Final log likelihood =", -l
# predict the test data
test_pred = map(lambda x: 0 if x<0.5 else 1, [sigmoid(inner(X, theta)) for X in test_x])
test_pair = np.vstack((np.array(test_lab), test_pred))
test_correct = filter(lambda x: x[0] == x[1], test_pair.T)
print "Accuracy = ", len(test_correct) * 1.0 / len(test_pred)

ll = -1697.51744519
ll = -143197.801827
ll = -8508.90496218
ll = -1662.30635
ll = -1640.40734323
ll = -1640.03672607
ll = -1639.03710526
ll = -1636.45342202
ll = -1630.7647578
ll = -1620.5379981
ll = -1608.46892967
ll = -1600.12028955
ll = -1597.25108903
ll = -1596.43103401
ll = -1594.6609878
ll = -1590.8669624
ll = -1582.36652552
ll = -1568.38645219
ll = -1573.12710035
ll = -1563.12188538
ll = -1730.8662104
ll = -1553.14766525
ll = -1551.33467062
ll = -1550.99356131
ll = -1550.98432802
ll = -1550.94773519
ll = -1550.80680828
ll = -1550.33015453
ll = -1546.68468455
ll = -1540.63459947
ll = -1535.30077889
ll = -1524.47917737
ll = -1515.22289358
ll = -1500.21666146
ll = -1493.66443366
ll = -1492.36234834
ll = -1492.23495489
ll = -1504.29563017
ll = -1492.23129908
ll = -1492.21829047
ll = -1492.11123077
ll = -1492.01427129
ll = -1491.92385856
ll = -1491.49164082
ll = -1490.15516329
ll = -1487.33551334
ll = -1482.64462887
ll = -1464.11483057
ll = -1568.29935212
ll = -1460.46434623
ll = -14